<a href="https://colab.research.google.com/github/sappyb/Codes-Doc/blob/master/2d_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uv pip install -q --system numba-cuda==0.4.0
import os
from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

import numpy as np
from numba import cuda
import matplotlib.pyplot as plt

# Generate a synthetic grayscale image (gradient)
height, width = 512, 512
img_host = np.tile(np.arange(width, dtype=np.uint8), (height, 1))

# Copy image to device and allocate output
img_device_in = cuda.to_device(img_host)
img_device_out = cuda.device_array_like(img_host)

# CUDA kernel to invert a grayscale image
@cuda.jit
def invert_kernel(in_img, out_img):
    x, y = cuda.grid(2)
    if x < in_img.shape[0] and y < in_img.shape[1]:
        out_img[x, y] = 255 - in_img[x, y]

# Define 2D blocks and grids
threads_per_block = (16, 16)
blocks_per_grid_x = (height + threads_per_block[0] - 1) // threads_per_block[0]
blocks_per_grid_y = (width  + threads_per_block[1] - 1) // threads_per_block[1]
blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

# Launch kernel
invert_kernel[blocks_per_grid, threads_per_block](img_device_in, img_device_out)
cuda.synchronize()

# Copy result back to host
img_inverted = img_device_out.copy_to_host()

# Display original and inverted images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img_host, cmap='gray')
axes[0].set_title("Original Image")
axes[0].axis('off')
axes[1].imshow(img_inverted, cmap='gray')
axes[1].set_title("Inverted Image")
axes[1].axis('off')
plt.show()

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: